In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools

from scipy.optimize import curve_fit, fsolve
from scipy.special import gamma
from scipy.special import gammaincc

CMTOMICRON = 1e4

#from runDataClass.py import runData

In [ ]:
def createBundleSizeList(start, end):
    """
    Inputs: two integers that signify the start and end of a series of simulations

    Returns: a list of values for the field bundle radius in the given simulations.
    """
    fieldBundles = pd.DataFrame({'Field Bundle Size': []})
    runNum = start

    while runNum <= end:
        
        simData = readRootTrees(runNum)
        for key in simData:
            globals()[key[:-6]] = simData[key]
        
        size = fieldBundles.shape[0]
        fieldBundles.loc[size] = [calcBundleSize(fieldLineData, metaData)['Middle']]
        runNum += 1
    return fieldBundles

In [ ]:
def calcBundleSize(Efield = pd.DataFrame(), geometry = pd.DataFrame(), check = False):
    """
    Inputs: Pandas dataframe for the electric field, pandas dataframe for the geometry (metaData), and an optional
    boolean 

    Returns: dictionary of three float values
    
    Uses a dataframe of electric field line points to first cut the data to the region of interest, then identify
    the outermost line in the set, and lastly calculate the distance of the midpoint of that line to the central
    axis. The boolean allows the user to plot the selected datapoint along with the field lines in order to verify
    that it has been chosen correctly.
    """

    try:
        zCut = Efield[Efield['Field Line z'] < 0 ]
        yzCut = zCut[zCut['Field Line y'] < geometry.iloc[0, 6]]
        xyzCut = yzCut[yzCut['Field Line x'] < geometry.iloc[0, 6]]
        
        lastRow = xyzCut.shape[0] - 1
        lastLineID = xyzCut['Field Line ID'].iloc[lastRow]
        
        
        lastLineCut = xyzCut[xyzCut['Field Line ID'] == lastLineID]

        cutLastRow = lastLineCut.shape[0] - 1
        
        midX = lastLineCut.iloc[int(cutLastRow/2), 1]
        midY = lastLineCut.iloc[int(cutLastRow/2), 2]
        midZ = lastLineCut.iloc[int(cutLastRow/2), 3]

        midBundleSize = math.sqrt(midX*midX + midY*midY)

        bottomX = lastLineCut.iloc[int(cutLastRow), 1]
        bottomY = lastLineCut.iloc[int(cutLastRow), 2]
        bottomZ = lastLineCut.iloc[int(cutLastRow), 3]

        bottomBundleSize = math.sqrt(bottomX*bottomX + bottomY*bottomY)
        
        topX = lastLineCut.iloc[0, 1]
        topY = lastLineCut.iloc[0, 2]
        topZ = lastLineCut.iloc[0, 3]

        topBundleSize = math.sqrt(topX*topX + topY*topY)
        
        bundleSize = {'Top': topBundleSize, 'Middle': midBundleSize, 'Bottom': bottomBundleSize}
        
        if check:
            plt.scatter(xyzCut['Field Line x'], xyzCut['Field Line z'], color = 'blue')
            plt.scatter(midX, midZ, color = 'yellow')
        
        return bundleSize

    except:
        try:
            if Efield.empty:
                print('Fieldline dataframe is empty')
        except:
            print('Invalid fieldline data type')

        try:
            if params.empty:
                print('Geometry dataframe is empty')
        except:
            print('Invalid Geometry data type')

In [ ]:
def createParamList(start, end, param):
    """
    Inputs: starting integer, ending integer, and the metaData title (string type) for a variable of interest.

    Returns: list

    Searches through a range of simulation data files to create a list of the values of the given variable
    throughout that range.
    """

    indVariab = pd.DataFrame({f'{param}': []})
    runNum = start

    while runNum <= end:
        
        simData = readRootTrees(runNum)
        for key in simData:
            globals()[key[:-6]] = simData[key]
        size = indVariab.shape[0]
        indVariab.loc[size] = [metaData[f'{param}'][0]]
        runNum += 1

    return indVariab

In [ ]:
def setScale(independent, dependent, xScale, yScale):
    """
    Inputs: two lists (or pandas dataframes) and two integers. 
    
    Returns: a dictionary with a list of data values for the x-axis, a list of data values for 
    the y-axis, the startinng value for x, and the starting value for y. 
    
    All four dictionary items intended for use in a matplot.lib xticks() or yticks() command.
    """
    try:
        xMin = independent.min(axis = 0)
        xMax = independent.max(axis = 0)
        yMin = dependent.min(axis = 0)
        yMax = dependent.max(axis = 0)
    
    except:
        xMin = min(independent)
        xMax = max(independent)
        yMin = min(dependent)
        yMax = max(dependent)
    
    xPlotMin = xScale*round(xMin/xScale, 0)
    xPlotMax = xMax+ xScale
    yPlotMin = yScale*round(yMin/yScale, 0)
    yPlotMax = yMax + yScale
    
    xTicks = np.arange(xPlotMin, xPlotMax, xScale)
    yTicks = np.arange(yPlotMin, yPlotMax, yScale)
    
    scale = {
    'xPlot': xTicks,
    'yPlot': yTicks,
    'xMin': xPlotMin,
    'yMin': yPlotMin,
    }

    return scale

In [ ]:
def multiPlotSimRange(dataStart, dataEnd, xaxis, yaxis, legend, colour, xScale, yScale, xUnits = '(\u03bcm)', yUnits = '(\u03bcm)', save = False):
    """
    Inputs: first and last sim data file numbers, the metaData title for the x-axis and the y-axis, the labels for the graph legend, 
    the color of the data set, the desired intervals for the x and y axes, the units for both axes, and a boolean for if you wish to 
    save the plot.

    Saves the plot under a directory labeled Plots with a name of the form y_vs_x.png.
    micro symbol unicode is: \u03bc
    """

    if not (len(dataStart) == len(dataEnd) == len(xaxis) == len(yaxis)):
        raise ValueError('Inputs must have the same length')
    
    if not os.path.exists('./Plots'):
            os.mkdir('./Plots')

    setNum = 0
    xExtremes = []
    yExtremes = []
    
    while setNum < len(dataStart):
        independent = createParamList(dataStart[setNum], dataEnd[setNum], xaxis[setNum])[xaxis[setNum]]
        if xUnits[setNum] == '(\u03bcm)':
            independent = independent*CMTOMICRON
        
        if yaxis[setNum] == 'Field Bundle Size':
            dependent = createBundleSizeList(dataStart[setNum], dataEnd[setNum])[yaxis[setNum]]   
        else:
            dependent = createParamList(dataStart[setNum], dataEnd[setNum], yaxis[setNum])[yaxis[setNum]] 
        if yUnits[setNum] == '(\u03bcm)':
            dependent = dependent*CMTOMICRON

        
        xExtremes.extend([independent.min(axis = 0), independent.max(axis=0)])
        yExtremes.extend([dependent.min(axis = 0), dependent.max(axis=0)])
        
        plt.scatter(independent, dependent, marker = 'x', s = 4, label = legend[setNum], color = colour[setNum])
        
        setNum += 1
    
    setNum -= 1
    
    Scales = setScale(xExtremes, yExtremes, xScale[setNum], yScale[setNum])
    plt.title(f'{yaxis[setNum]} vs {xaxis[setNum]}')
    plt.legend(loc = 'upper left')
    plt.xlabel(f'{xaxis[setNum]} {xUnits[setNum]}')
    plt.xticks(Scales['xPlot'])
    plt.ylabel(f'{yaxis[setNum]} {yUnits[setNum]}')
    plt.yticks(Scales['yPlot'])

    if save[setNum]:
        plt.savefig(f'./Plots/{yaxis[setNum]}_vs_{xaxis[setNum]}.png')
    plt.show()

In [ ]:
'''
multiPlotSimRange(
    [1347, 1384],
    [1383, 1499],
    ['Pitch', 'Pitch'],
    ['Field Bundle Size', 'Field Bundle Size'],
    ['Radius = 35 (\u03bcm)', 'Radius = 55 (\u03bcm)'],
    ['blue', 'red'],
    [10, 10],
    [2, 2],
    ['(\u03bcm)', '(\u03bcm)'],
    ['(\u03bcm)', '(\u03bcm)'],
    [False, False]
)
'''

In [ ]:
## Plot e;ectron tracks in avalanches
if __name__ == '__main__':
    metaData = allData[runNumbers[0]]['metaData']
    
    allElectrons = allData[runNumbers[0]]['electronData']
    allElectronTracks = allData[runNumbers[0]]['electronTrackData']

    plotSingleAvalanche(allElectronTracks, 0, metaData, 'First')

    allAvalanches = allData[runNumbers[0]]['avalancheData']
    largestAvalanche = allAvalanches['Total Electrons'].max()
    
    largestAvalancheID = allAvalanches[allAvalanches['Total Electrons']==largestAvalanche]['Avalanche ID'].iloc[0]
    plotSingleAvalanche(allElectronTracks, largestAvalancheID, metaData, 'Largest')
    
    longestZ = -1
    longestID = -1
    for i in range(len(allAvalanches[allAvalanches['Total Electrons'] == 1])):
        avalancheID = allAvalanches[allAvalanches['Total Electrons'] == 1]['Avalanche ID'].iloc[i]
    
        inAvalanche  = allElectrons[allElectrons['Avalanche ID'] == avalancheID]
        x = inAvalanche['Final x'].iloc[0]
        y = inAvalanche['Final y'].iloc[0]
        z = inAvalanche['Final z'].iloc[0]
        stat = inAvalanche['Exit Status'].iloc[0]
    
        halfPitch = .99*metaData['Pitch'].iloc[0]/2.#TODO - not square geometry here anymore
        if stat == -7:
            continue
        elif np.abs(x) >= halfPitch or np.abs(y) >= halfPitch:
            if np.abs(z) > longestZ:
                longestID = avalancheID
                longestZ = np.abs(z)
    
    plotSingleAvalanche(allElectronTracks, longestID, metaData, 'Furthest Drift')

In [ ]:
def plotAvalancheSize(avalancheData, metaData, binWidth=1) -> None:
    """
    """
    runNo = metaData['runNo'].iloc[0]

    # Make plot for all avalanches
    gain = avalancheData['Total Electrons'].mean()

    bins = np.arange(avalancheData['Total Electrons'].min(), 
                     avalancheData['Total Electrons'].max()+1, 
                     binWidth)

    fig = plt.figure(figsize=(8, 5))
    fig.suptitle(f'All Avalanche Sizes: Run {runNo}')
    
    ax = fig.add_subplot(211)
    
    ax.hist(avalancheData['Total Electrons'], 
             bins=bins, label='Simulation')
    
    ax.axvline(x=gain, 
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('')
    ax.legend()
    ax.grid()

    # Remove avalanches that have 1 electron or reached limit
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]
    
    gain = trimmedAvalanche['Total Electrons'].mean()
    ## Note to self: 
    #     Removing 1e avalanches will increase the average value
    #     Removing the limit-hit ones will decrease the average

    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)

    fig = plt.figure(figsize=(8, 5))
    fig.suptitle(f'Trimmed Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(211)
    
    ax.hist(trimmedAvalanche['Total Electrons'], 
             bins=bins, label='Simulation')
    
    ax.axvline(x=gain, 
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('')
    ax.legend()
    ax.grid()
    
    return
    

In [ ]:
def polya(n, nBar, theta):
    return (1/nBar)*np.power(theta+1, theta+1)/gamma(theta+1)*np.power(n/nBar, theta)*np.exp(-(n/nBar)*(theta+1))

def exponential(n, nBar):
    #This is a polya with theta=0
    return (1/nBar)*np.exp(-(n/nBar))

def fitPolya(avalancheData, metaData, binWidth=1):
    """
    """

    # Remove avalanches that have 1 electron or reached limit
        ##Note on 1e avalanches:
        ## These certainly can happen, however they are more likely that
        ## the electron has attached or drifted out of the volume.
 
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]

    gain = trimmedAvalanche['Total Electrons'].mean()

    if (gain < 5) or (gain > metaData['Avalanche Limit'].iloc[0]):
        print(f"Unable to do fit. Gain is: {gain:.2f}.")
        return None, None
    
    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(trimmedAvalanche['Total Electrons'], 
                             bins=bins)
    prob = counts/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [gain, 1.]
    bounds = ([1, 0], [metaData['Avalanche Limit'].iloc[0], 10])

    popt, pcov = curve_fit(polya, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perr = np.sqrt(np.diag(pcov))

    fitPolyaParam = {
        'nBar': popt[0],
        'nBarErr': perr[0],
        'theta': popt[1],
        'thetaErr': perr[1]
    }

    plotParam = {
        'gain': gain,
        'bins': bins,
        'binCenters': binCenters,
        'prob': prob
    }

    return fitPolyaParam, plotParam

def fitExponential(avalancheData, metaData, binWidth=1):
    """
    """

    # Remove avalanches that have 1 electron or reached limit
        ##Note on 1e avalanches:
        ## These certainly can happen, however they are more likely that
        ## the electron has attached or drifted out of the volume.
 
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]

    gain = trimmedAvalanche['Total Electrons'].mean()

    if (gain < 5) or (gain > metaData['Avalanche Limit'].iloc[0]):
        print(f"Unable to do fit. Gain is: {gain:.2f}.")
        return None, None
    
    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(trimmedAvalanche['Total Electrons'], 
                             bins=bins)
    prob = counts/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [gain]
    bounds = ([1], [metaData['Avalanche Limit'].iloc[0]])

    popt, pcov = curve_fit(exponential, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perr = np.sqrt(np.diag(pcov))

    fitExponentialParam = {
        'nBar': popt[0],
        'nBarErr': perr[0],
    }

    plotParam = {
        'gain': gain,
        'bins': bins,
        'binCenters': binCenters,
        'prob': prob
    }

    return fitExponentialParam, plotParam
    
def plotAvalancheFits(avalancheData, metaData, binWidth=1):
    """
    """
    runNo = metaData['runNo'].iloc[0]
    gainRaw = avalancheData['Total Electrons'].mean()

    fitPolyaParam, plotParam = fitPolya(avalancheData, metaData, binWidth)
    
    polyaFit = polya(plotParam['binCenters'], fitPolyaParam['nBar'], fitPolyaParam['theta'])

    fitExponentialParam, _ = fitExponential(avalancheData, metaData, binWidth)
    
    exponentialFit = exponential(plotParam['binCenters'], fitExponentialParam['nBar'])
    
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f'Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(111)
    
    ax.bar(plotParam['bins'][:-1], plotParam['prob'], 
           width=binWidth, align='edge', label='Simulated Data', color='b')
    ax.axvline(x=gainRaw, 
               c='g', ls='--', label=f'Simulated Gain (Raw) = {gainRaw:.1f}e')
    ax.axvline(x=plotParam['gain'], 
               c='g', ls=':', label=f'Simulated Gain (Trimmed) = {plotParam['gain']:.1f}e')

    ax.plot(plotParam['binCenters'], polyaFit, 
            'm-', lw=2, label=r'Fitted Polya ($\theta$' + f' = {fitPolyaParam['theta']:.4f})')
    ax.axvline(x=fitPolyaParam['nBar'], 
               c='m', ls=':', label=f'Fitted Gain = {fitPolyaParam['nBar']:.1f}e')

    ax.plot(plotParam['binCenters'], exponentialFit, 
            'r', lw=2, label=f'Fitted Exponential')
    ax.axvline(x=fitExponentialParam['nBar'], 
               c='r', ls=':', label=f'Fitted Gain = {fitExponentialParam['nBar']:.1f}e')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('Probability')
    
    ax.legend()
    ax.grid()

    return

In [ ]:
def polyaEfficiency(nBar, theta, threshold):
    s = theta+1
    x = s*threshold/nBar
    return gammaincc(s, x)
    
def getPolyaEfficiency(avalancheData, metaData, binWidth=1, threshold=10):
    """
    """

    fitPolyaParam, _ = fitPolya(avalancheData, metaData, binWidth)

    if fitPolyaParam is None:
        print(f"Unable to determine efficiency.")
        return 0, 0

    efficiency = polyaEfficiency(fitPolyaParam['nBar'], fitPolyaParam['theta'], threshold)

    nBarWithErr = [
        fitPolyaParam['nBar'] - fitPolyaParam['nBarErr'],
        fitPolyaParam['nBar'] + fitPolyaParam['nBarErr']
        ]
    thetaWithErr = [
        fitPolyaParam['theta'] - fitPolyaParam['thetaErr'], 
        fitPolyaParam['theta'] + fitPolyaParam['thetaErr']
        ]

    efficiencyWithErrors = np.zeros((2, 2))
    for i, n in enumerate(nBarWithErr):
        for j, t in enumerate(thetaWithErr):
            efficiencyWithErrors[i, j] = polyaEfficiency(n, t, threshold)

    efficiencyErr = [efficiencyWithErrors.max()-efficiency, efficiencyWithErrors.min()-efficiency]
    return efficiency, efficiencyErr

In [ ]:
## Plot avalanche sizes
if __name__ == '__main__':
    metaData = allData[runNumbers[0]]['metaData']
    allAvalanches = allData[runNumbers[0]]['avalancheData']

    plotAvalancheSize(allAvalanches, metaData, binWidth=5)
    plotAvalancheFits(allAvalanches, metaData, binWidth=5)

    eff, errs = getPolyaEfficiency(allAvalanches, metaData, binWidth=5, threshold=3)
    print(f"Efficiency: {eff:.4f}")
    print(f"Efficiency Range: {eff + errs[1]:.4f} - {eff + errs[0]:.4f}")

In [ ]:
def plotGeneralPolya():
    """
    """
    n = np.linspace(0, 4, 101)
    theta = np.arange(0, 5, 1)
    plt.figure(figsize=(6, 4))
    
    for t in theta:
        prob = polya(n, 1, t)
        plt.plot(n, prob,
                label=r'$\theta$'+f' = {t:.2f}')
        
    plt.title(f"General Polya Distribution")
    plt.xlabel(r'Avalanche Size ($n/\bar{n}$)')
    plt.ylabel(r'$\bar{n}$ x Probability')
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()

def plotPolya(theta):
    """
    """
    gain = [10, 25, 50, 75, 100]

    n = np.arange(0, 101, 1)
    
    numPlots = len(theta)
    numRows = int(np.ceil(numPlots/2))
    
    # Create the figure and add subplots
    fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 5*numRows))
    axesFlat = axes.flatten()

    fig.suptitle(f'Polya Avalanches')

    for i, t in enumerate(theta):
        for nBar in gain:
            prob = polya(n, nBar, t)
            axesFlat[i].plot(n, prob,
                             label=r'$\bar{n}$'+f' = {nBar:.0f}')

        axesFlat[i].set_title(r'$\theta$'+f' = {t:0.2f}')
        axesFlat[i].set_xlabel('Avalanche size')
        axesFlat[i].set_ylabel('Probability')
        axesFlat[i].legend()
        axesFlat[i].grid(True, alpha=0.5)

    for j in range(numPlots, len(axesFlat)):
        fig.delaxes(axesFlat[j])
    plt.show()

def plotPolyaEfficiency(theta):
    """
    """
    k = np.linspace(0, 1, 101) #Ratio: Threshold/Gain

    plt.figure(figsize=(12, 5))

    for t in theta:
        efficiency = gammaincc(t+1, k*(t+1))
        plt.plot(k, efficiency,
                 label=r"$\theta$"+f" = {t:0.2f}")

    targetEfficiency = 0.95
    plt.axhline(y=targetEfficiency,
                c='r', ls='--', label=f'{targetEfficiency*100:.0f}% Efficiency')
    plt.axvline(x=-np.log(targetEfficiency),
                c='r', ls=':', label=r'$\theta = 0$ Max: '+f'{-np.log(targetEfficiency):.3f}')

    plt.title(f"Efficiency of Polya: "
              +r"$\eta = \frac{\Gamma\left(\theta+1, (\theta+1)*n_{t}/\bar{n}\right)}{\Gamma\left(\theta+1\right)}$")
    plt.xlabel("Threshold / Gain Fraction: "
               +r"$n_{t} / \bar{n}$")
    plt.ylabel(f"Efficiency")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()

    return

def solvePolyEfficiency(nBar, theta, threshold, targetEff):

    calcEff = polyaEfficiency(nBar, theta, threshold)
    return targetEff - calcEff
    
def solveForGain(efficiency, theta, threshold):
    """
    """
    gainMin = 1
    gainSolved, _, _, mesg = fsolve(solvePolyEfficiency,
                                    gainMin, args=(theta, threshold, efficiency), full_output=True)
    #print(f'Gain Solution: {gainSolved[0]:.4f} ({mesg})')

    return gainSolved[0]

def plotThreshold():
    """
    """
    threshold = np.linspace(0, 16, 11)
    efficiency = [.95, .9]

    colors = ['b', 'r', 'g']

    plt.figure(figsize=(6, 4))
    
    for i, eff in enumerate(efficiency):
        gain = -threshold/np.log(eff)
        plt.plot(threshold, gain,
                 c=colors[i], label=f'Target Efficiency = {eff*100:.0f}%')

        theta5 = threshold*solveForGain(eff, 0.5, 1)
        plt.plot(threshold, theta5,
                 c=colors[i], ls=':', label=r'$\theta$ = 0.5')
        
        theta1 = threshold*solveForGain(eff, 1, 1)
        plt.plot(threshold, theta1,
                 c=colors[i], ls='--', label=r'$\theta$ = 1')

        theta2 = threshold*solveForGain(eff, 2, 1)
        plt.plot(threshold, theta2,
                 c=colors[i], ls='-.', label=r'$\theta$ = 2')

    plt.title(f"Minimum Gain Required to Achieve Efficiency")
    plt.xlabel('Detector Threshold')
    plt.ylabel('Gain')
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()


In [ ]:
def plotPolyExamples(thetaStart, thetaEnd, numSteps):
    """
    """
    theta = np.linspace(thetaStart, thetaEnd, numSteps)

    plotGeneralPolya()
    plotPolya(theta)
    plotPolyaEfficiency(theta)
    plotThreshold()

if __name__ == '__main__':
    plotPolyExamples(0, 5, 6)

In [ ]:
def plotScannedPolyaParam(runNumbers):
    """
    """
    allData = readMultipleRuns(runNumbers, ['avalancheData'])
    metaData = allData['metaData']


    holeRadius = []
    standoff = []
    polyaGain = []
    polyaTheta = []
    polyaGainErr = []
    polyaThetaErr = []
    
    for inRun in runNumbers:
        fitPolyaParam, _ = fitPolya(allData[f'{inRun}']['avalancheData'], metaData[f'{inRun}'])

        holeRadius.append(metaData[f'{inRun}']['Hole Radius'].iloc[0]*CMTOMICRON)
        standoff.append(metaData[f'{inRun}']['Grid Standoff'].iloc[0]*CMTOMICRON)
        polyaGain.append(fitPolyaParam['nBar'])
        polyaGainErr.append(fitPolyaParam['nBarErr'])
        polyaTheta.append(fitPolyaParam['theta'])
        polyaThetaErr.append(fitPolyaParam['thetaErr'])
        

    fig3D = plt.figure(figsize=(12, 6))
    fig3D.suptitle(f'Test')
    
    ax3D1 = fig3D.add_subplot(121, projection='3d')
    ax3D2 = fig3D.add_subplot(122, projection='3d')

    ax3D1.errorbar(holeRadius, standoff, polyaGain,
                 zerr=polyaGainErr, ls='', marker='o')
    ax3D2.errorbar(holeRadius, standoff, polyaTheta,
                 zerr=polyaThetaErr, ls='', marker='o')

    ax3D1.set_xlabel('Hole Radius (um)')
    ax3D1.set_ylabel('Standoff (um)')
    ax3D1.set_zlabel('Polya Gain')

    ax3D2.set_xlabel('Hole Radius (um)')
    ax3D2.set_ylabel('Standoff (um)')
    ax3D2.set_zlabel('Polya Theta')


    return

#trialARuns = list(range(5209, 5218 + 1))
#plotScannedPolyaParam(trialARuns)